In [1]:
import pandas as pd
import re
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [2]:
train_data = pd.read_csv("../../Downloads/6e35ef66b3d211e7/train.csv")


In [3]:
test_data = pd.read_csv("../../Downloads/6e35ef66b3d211e7/test.csv")


In [4]:
def text_process(mess):
    mess = re.sub("[^a-zA-Z]", " ", str(mess).lower())
    list_cell_content = str(mess).split()
    cell_content = [lmtzr.lemmatize(word) for word in list_cell_content]
    cell_content = " ".join(cell_content)
    return cell_content

def data_preprocess(data):
    data = data.fillna("")
    data['title_words'] = data['name'].apply(lambda x : len(str(x).split(" ")))
    data['des_words'] = data['desc'].apply(lambda x : len(str(x).split(" ")))
    data['duration_c'] = data['deadline'] - data['created_at']
    data['duration_l'] = data['deadline'] - data['launched_at']
    data['duration'] = data['launched_at'] - data['created_at']
    data['dead_state'] = data['deadline'] - data['state_changed_at']
    data['desc'] = data.desc.apply(lambda x: text_process(x))
    data['keywords'] = data.keywords.apply(lambda x: text_process(x))
    data['name'] = data.name.apply(lambda x: text_process(x))
    data['text'] = data['desc'] + data['keywords'] + data['name']
    return data

In [5]:
train_data = data_preprocess(train_data)

In [6]:
test_data = data_preprocess(test_data)

In [7]:
result = train_data['final_status']
# data = data[['name','desc','goal','duration','title_words','des_words']]

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
vectorizer = TfidfVectorizer(stop_words='english',max_df=0.9,min_df=55)

In [245]:
from sklearn.model_selection import train_test_split

In [246]:
X_train, X_test, y_train, y_test = train_test_split(
...     train_data,result, test_size=0.55, random_state=42)

In [247]:
# vectorizer.fit_transform(X_train['desc'].values)

In [21]:

from sklearn_pandas import DataFrameMapper

from sklearn.preprocessing import LabelEncoder
mapper = DataFrameMapper([
     ('text', vectorizer),
     ('goal',None),
     ('duration_c',None),
    ('dead_state',None),
    ('disable_communication',LabelEncoder()),
    ('duration',None),
    ('duration_l',None),
    ('currency',CountVectorizer()),
     ('country',CountVectorizer()),
     ])

In [10]:
feature = mapper.fit_transform(train_data)
test = mapper.transform(test_data)
# final_test = mapper.transform(test_data)

In [22]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.svm import SVC
model = RandomForestClassifier(
 n_estimators = 300)
# model =  SVC(kernel='poly', degree=3, C=1.0)
feature.shape

(108129, 3433)

In [23]:
model.fit(feature,result
         )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
# a = accuracy_score(y_test,model.predict(test))

# print (a)
ans = model.predict(test)

In [26]:
test_df = test_data

In [27]:
test_df['final_status'] = pd.DataFrame(ans)

In [28]:
df = test_df[['project_id','final_status']]

In [29]:
df[df.project_id == "kkst1000002330"]

,project_id,final_status
23165,kkst1000002330,0


In [30]:
df.to_csv("submission16.csv",index=False)

In [ ]:
test_data.columns

In [211]:
train_data.columns

Index(['project_id', 'name', 'desc', 'goal', 'keywords',
       'disable_communication', 'country', 'currency', 'deadline',
       'state_changed_at', 'created_at', 'launched_at', 'backers_count',
       'final_status', 'title_words', 'des_words', 'duration_c', 'duration_l',
       'duration', 'dead_state', 'text'],
      dtype='object')

In [ ]:
train